<a href="https://www.kaggle.com/code/hilalalpak/ab-testing-analysis?scriptVersionId=181064098" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="color: #ffffff; border-radius: 0 50px 0 20px; background-color:#3b5998; padding: 8px; margin-bottom: 10px;"> 
    <p style="margin: 0; font-size: 30px; text-align: center; font-family: Arial, sans-serif; font-weight: bold; letter-spacing: 2px;"> AB Testing - Comparing the Conversion of Bidding Methods</p> 
</div>

<div style="display:inline-block; text-align:left; margin-bottom:10px;">
    <div style="border: 2px solid #333333; border-radius: 0 50px 0 0; padding: 10px; margin-bottom: 10px; background-color:#232f3e;">
        <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
            <p style="font-weight:bold; color:white; margin:0; font-size: 150%;font-family: Arial, sans-serif"> Examine Dataset </p>
        </blockquote> 
        <p style="color:#ffffff; font-size: 110%; font-family: Arial, sans-serif; margin-top:5px;">The primary challenges in e-commerce include accurately assessing product ratings post-purchase and sorting product reviews effectively. Solving these issues is essential for enhancing customer satisfaction, increasing product visibility, and ensuring seamless shopping experiences. This, in turn, leads to sales growth for e-commerce platforms and sellers while facilitating smooth customer purchasing journeys.</p>
    </div>   
</div>


   
        
<div style="display:inline-block; text-align:left; margin: auto;">
    <div style="border: 2px solid #333333; border-radius: 0 50px 0 0; padding: 10px; margin-bottom: 10px; background-color:#232f3e;">
         <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
            <p style="font-weight:bold; color:white; margin:0; font-size: 150%;font-family: Arial, sans-serif"> Business Problem </p>
        </blockquote> 
        <p style="color:#ffffff; font-size: 110%; font-family: Arial, sans-serif; margin-top:5px;">Recently, Facebook introduced a new bidding type called "average bidding" as an alternative to the existing "maximum bidding" method. With this innovation, companies like x.com, one of our clients, are planning to conduct an A/B test to evaluate whether "average bidding" yields more conversions compared to the "maximum bidding" approach. </p>
    </div>
</div>


<div style="background-color:#E3F2FD; padding:30px; width: 400px; margin: auto; border-radius: 30px 0 30px 0; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    <h2 style="text-align: center; color:#333333; font-size: 18px; margin-bottom: 15px;">Variables</h2>
    <div style="margin-bottom: 10px; white-space: nowrap;">
        <span style="font-weight:bold; color:#333333;">Impression:</span> <span style="color:#333333;">Ad views</span>
    </div>
    <div style="margin-bottom: 10px; white-space: nowrap;">
        <span style="font-weight:bold; color:#333333;">Click:</span> <span style="color:#333333;">Indicates the number of times the ad was clicked.</span>
    </div>
    <div style="margin-bottom: 10px; white-space: nowrap;">
        <span style="font-weight:bold; color:#333333;">Buy:</span> <span style="color:#333333;">The number of products purchased after clicking on the ads</span>
    </div>
    <div style="margin-bottom: 10px; white-space: nowrap;">
        <span style="font-weight:bold; color:#333333;">Earnings:</span> <span style="color:#333333;">Earnings after purchased items</span>
    </div>       
</div>

  
<div style="display:inline-block; text-align:left; margin: auto;">
        <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
            <p style="color:white; font-size: 100%; font-family: Arial, sans-serif; margin-top:5px; padding-left: 15px;">  
                A/B testing (split testing) is a method of comparing two versions of a web page or application against each other to determine which performs better. AB testing mainly uses statistical analysis to determine which variation performs better for a given conversion goal.
            </p>
        </blockquote>
   </div>
</div>



In [1]:
import sys
import subprocess

def install_package(package):
    process = subprocess.Popen([sys.executable, '-m', 'pip', 'install', package],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

install_package('statsmodels')

In [2]:
# import Required Libraries

import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
import warnings
import sys
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal

from statsmodels.stats.proportion import proportions_ztest

if not sys.warnoptions:
    warnings.simplefilter("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
control = pd.read_excel("/kaggle/input/ab-testing/ab_testing.xlsx", sheet_name="Control Group")
test = pd.read_excel("/kaggle/input/ab-testing/ab_testing.xlsx", sheet_name="Test Group")

In [4]:
control.head()
print("MISSING VALUES".center(50, "-"))
control.isnull().sum()
print("DUPLICATED VALUES".center(50, "-"))
control.duplicated().sum()
print("INFO".center(50, "-"))
control.info()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


------------------MISSING VALUES------------------


Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

----------------DUPLICATED VALUES-----------------


0

-----------------------INFO-----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [5]:
test.head()
print("MISSING VALUES".center(50, "-"))
test.isnull().sum()
print("DUPLICATED VALUES".center(50, "-"))
test.duplicated().sum()
print("INFO".center(50, "-"))
test.info()

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


------------------MISSING VALUES------------------


Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

----------------DUPLICATED VALUES-----------------


0

-----------------------INFO-----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [6]:
# Explain the hypothesis
# H0 : M1 = M2 ----- H1 : M1 != M2

control["Purchase"].mean()
test["Purchase"].mean()

550.8940587702316

582.1060966484677

<div style="background-color: #333333; border-radius: 8px; padding: 1px; margin-bottom: 3px;">
    <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
    <p style="margin: 0; font-size: 12px; text-align: left; font-family: Arial, sans-serif; font-weight: bold; letter-spacing: 1px; color: #ffffff;"> When considering the mean value between the observation and control groups, there seems to be a difference. However, to verify whether this difference occurred by chance or if it also exists statistically, we will conduct an AB Test. </p> 
    </blockquote>
</div>

<div style="background-color: #333333; border-radius: 8px; padding: 1px; margin-bottom: 3px;">
    <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: Arial, sans-serif; font-weight: bold; letter-spacing: 1px; color: #ffffff;"> We will check the assumptions of normality and homogeneity of variance. </p> 
    </blockquote>
</div>

In [7]:
test_stat, pvalue = shapiro(control["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# P-value > 0.05. Therefore, H0 cannot be rejected.The normality assumption is met.

#`test_stat` variable indicates the group difference magnitude, and `pvalue` determines its statistical significance.

Test Stat = 0.9773, p-value = 0.5891


In [8]:
test_stat, pvalue = shapiro(test["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# P-value > 0.05. Therefore, H0 cannot be rejected.The normality assumption is met.

Test Stat = 0.9589, p-value = 0.1541


In [9]:
test_stat, pvalue = levene(control["Purchase"], test["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# P-value > 0.05. Therefore, H0 cannot be rejected.The assumption of homogeneity of variances is met.

Test Stat = 2.6393, p-value = 0.1083


<div style="background-color: #333333; border-radius: 20px; padding: 5px; margin-bottom: 6px; text-align: right;">
    <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
        <p style="margin: 0; font-size: 14px; text-align: left; font-family: 'Arial', sans-serif; font-weight: bold; letter-spacing: 1px; color: #ffffff;">If One Assumption is Met while the Other is Not:</p>
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: 'Arial', sans-serif; letter-spacing: 1px; color: #ffffff;">If the normality assumption is met but the assumption of homogeneity of variances is not, alternative parametric tests such as Welch's T-Test or unequal variance t-test can be used.</p>
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: 'Arial', sans-serif; letter-spacing: 1px; color: #ffffff;">If the assumption of homogeneity of variances is met but the normality assumption is not, non-parametric tests such as the Mann-Whitney U Test or Kruskal-Wallis Test can be employed.</p>
    </blockquote>
    <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
        <p style="margin: 0; font-size: 14px; text-align: left; font-family: 'Arial', sans-serif; font-weight: bold; letter-spacing: 1px; color: #ffffff;">If Neither Assumption is Met:</p>
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: 'Arial', sans-serif; letter-spacing: 1px; color: #ffffff;">Data transformation or non-parametric tests (Mann-Whitney U Test or Kruskal-Wallis Test) can be utilized.</p>
    </blockquote>
    <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
        <p style="margin: 0; font-size: 14px; text-align: left; font-family: 'Arial', sans-serif; font-weight: bold; letter-spacing: 1px; color: #ffffff;">If Both Assumptions are Met:</p>
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: 'Arial', sans-serif; letter-spacing: 1px; color: #ffffff;">Independent two-sample t-test.</p>
    </blockquote>
</div>


In [10]:
# Independent two-sample t-test.

test_stat, pvalue = ttest_ind(control["Purchase"],
                              test["Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


<div style="background-color: #E3F2FD; border-radius: 8px; padding: 1px; margin-bottom: 3px;">
    <blockquote style="border-left: 5px solid #FFA500; padding-left: 10px;"> 
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: Arial, sans-serif; font-weight: bold; letter-spacing: 1px; color: black;"> The obtained p-value of 0.3493 suggests that there is no statistically significant difference between the purchase averages of the control and test groups. In other words, there is not enough evidence to determine whether average bidding brings more conversions compared to maximum bidding. </p> 
    </blockquote>
</div>

<div style="background-color:#E3F2FD; border-radius: 8px; padding: 10px; margin-bottom: 3px;">
        <p style="margin: 0; font-size: 12px; text-align: left; font-family: Arial, sans-serif; font-weight: bold; letter-spacing: 1px; color: black;"> Considering that the average purchase value of the test group is higher than that of the control group, there is a higher probability that average bidding may yield more conversions compared to maximum bidding. In this case, I would recommend to the client to continue the test and gather more data for the new bidding strategy.</p> 
</div>